In [1]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
from neurorient import NeurOrient

In [2]:
from mpi4py import MPI

In [3]:
spi_data = torch.load('/pscratch/sd/z/zhantao/neurorient_repo/data/1bxr_randquat.pt')
model_dir = '/pscratch/sd/z/zhantao/neurorient_repo/model'
print(spi_data.keys())

dict_keys(['orientations', 'images', 'pixel_position_reciprocal', 'volume', 'img_real_mesh', 'vol_real_mesh', 'time_stamp'])


In [4]:
# model = NeurOrient.load_from_checkpoint('/pscratch/sd/z/zhantao/neurorient_repo/model/lightning_logs/version_2/checkpoints/last.ckpt')

In [5]:
model = NeurOrient(spi_data['pixel_position_reciprocal'], volume_type='intensity', path=model_dir)
# model.to('cpu');

In [6]:
# model.compute_autocorrelation()
# orientations = model.image_to_orientation(spi_data['images'][:10].unsqueeze(1))
# from neurorient.reconstruction.slicing import get_rho_function
# rho_true = torch.fft.fftshift(torch.fft.ifftn(torch.fft.ifftshift(spi_data['volume']))).real.clip(0)
# rho_func = get_rho_function(spi_data['vol_real_mesh'], rho_true)
# rho_img_coords = torch.from_numpy(rho_func(spi_data['img_real_mesh'].numpy()))
# rho_true = torch.from_numpy(rho_func(model.grid_position_real.numpy()))
# ac = model.compute_autocorrelation(rho=rho_true)
# slices = model.gen_slices(ac, orientations)
# plt.imshow(slices.detach().numpy()[0].clip(0), vmax=slices.max() * 0.001)

In [7]:
import lightning as L

In [8]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [9]:
dataset = TensorDataset(spi_data['intensities'].unsqueeze(1))
dataloader = DataLoader(dataset, batch_size=50, shuffle=True)

In [10]:
# batch = next(iter(dataloader))
# # model.training_step(batch, 0)
# slices_true = batch[0]
# orientations = model.image_to_orientation(slices_true)

# from neurorient.reconstruction.slicing import gen_nonuniform_normalized_positions
# # get reciprocal positions based on orientations
# HKL = gen_nonuniform_normalized_positions(
#     orientations, model.pixel_position_reciprocal, model.over_sampling)
# slices_pred = model.predict_slice(HKL)

In [11]:
from neurorient.utils_visualization import display_images_in_parallel, display_volumes

# display_images_in_parallel(torch.randn(10, 3,3), torch.randn(10, 3,3))
# display_volumes(torch.randn(10,3,3,3))

In [12]:
from lightning.pytorch.callbacks import ModelCheckpoint, TQDMProgressBar

checkpoint_callback = ModelCheckpoint(
    every_n_train_steps=10, save_last=True, save_top_k=1, monitor="train_loss"
)

In [13]:
trainer = L.Trainer(
    max_epochs=100, accelerator='gpu',
    callbacks=[checkpoint_callback, TQDMProgressBar(refresh_rate=10)],
    log_every_n_steps=1, devices=torch.cuda.device_count(),
    enable_checkpointing=True, default_root_dir=model.path)
trainer.fit(model, dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type          | Params
--------------------------------------------------------
0 | orientation_predictor | ResNet2Rotmat | 11.2 M
1 | volume_predictor      | IntensityNet  | 528 K 
2 | nufft_forward         | KbNufft       | 0     
--------------------------------------------------------
11.7 M    Trainable params
0         Non-trainable params
11.7 M    Total params
46.834    Total estimated model params size 

Epoch 0:   0%|          | 0/200 [00:00<?, ?it/s] 

/global/homes/z/zhantao/.local/lib/python3.9/site-packages/matplotlib/image.py:446: RuntimeWarning: invalid value encountered in double_scalars
  newmin = vmid - dv * fact


Epoch 0:  45%|████▌     | 90/200 [01:26<01:45,  1.04it/s, v_num=1.45e+7]